# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Уменьшение размеров модели
### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 баллов__.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн 29.11.24 23:59__ \
__Жесткий дедлайн 2.12.24 23:59__

### О задании

В этом задании вам предстоит научиться решать задачу Named Entity Recognition (NER) на самом популярном датасете – [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). В вашем распоряжении будет предобученный BERT, который вам необходимо уменьшить с минимальными потерями в качестве до размера 20М параметров. Для этого вы самостоятельно реализуете факторизацию эмбеддингов, дистилляцию, шеринг параметров и так далее.

В этом задании вам придется проводить довольно много экспериментов, поэтому мы рекомендуем не писать весь код в тетрадке, а завести разные файлы для отдельных логических блоков и скомпоновать все в виде проекта. Это позволит вашему ноутбуку не разрастаться и сильно облегчит задачу и вам, и проверяющим. Так же постарайтесь логгировать все ваши эксперименты в wandb, чтобы ничего не потерялось.

### Оценивание
Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором, как и раньше, от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета можно потерять баллы за сами задания. Задания делятся на две части: _номерные_ и _на выбор_. За _номерные_ можно получить в сумме 6 баллов, за задания _на выбор_ можно получить до 16. То есть за все дз можно получить 24 балла. Все, что вы наберете свыше 10, будет считаться бонусами.


### О датасете

Named Entity Recognition – это задача классификации токенов по классам сущностей. В CoNLL-2003 для именования сущностей используется маркировка **BIO** (Beggining, Inside, Outside), в которой метки означают следующее:

- *B-{метка}* – начало сущности *{метка}*
- *I-{метка}* – продолжнение сущности *{метка}*
- *O* – не сущность

Существуют так же и другие способы маркировки, например, BILUO. Почитать о них можно [тут](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)) и [тут](https://www.youtube.com/watch?v=dQw4w9WgXcQ).

Всего в датасете есть 9 разных меток.
- O – слову не соответствует ни одна сущность.
- B-PER/I-PER – слово или набор слов соответстует определенному _человеку_.
- B-ORG/I-ORG – слово или набор слов соответстует определенной _организации_.
- B-LOC/I-LOC – слово или набор слов соответстует определенной _локации_.
- B-MISC/I-MISC – слово или набор слов соответстует сущности, которая не относится ни к одной из предыдущих. Например, национальность, произведение искусства, мероприятие и т.д.

Приступим!

Начнем с загрузки и предобработки датасета.

In [28]:
!git clone https://github.com/aapetukhov/NLP.git

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Cloning into 'NLP'...
^C


In [29]:
%cd NLP/nlp_hw4

[Errno 2] No such file or directory: 'NLP/nlp_hw4'
/kaggle/working


In [2]:
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from datasets import DatasetDict
from seqeval.metrics import f1_score, classification_report
from tqdm import tqdm

from datasets import DatasetDict
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from transformers.tokenization_utils import BatchEncoding
from transformers import DistilBertConfig, DistilBertForTokenClassification, BertConfig, BertForTokenClassification

In [4]:
dataset = load_dataset("eriktks/conll2003")

dataset = dataset.remove_columns(["id", "pos_tags", "chunk_tags"])

label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
label_to_id = {label: idx for idx, label in enumerate(label_names)}
id_to_label = {idx: label for idx, label in enumerate(label_names)}

# alignment sample
words = dataset["train"][0]["tokens"]
labels = dataset["train"][0]["ner_tags"]

for i in range(len(words)):
    print(f'{words[i]}\t{label_names[labels[i]]}')

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O


### Предобработка

На протяжении всего домашнего задания мы будем использовать _cased_ версию BERT, то есть токенизатор будет учитывать регистр слов. Для задачи NER регистр важен, так как имена и названия организаций или предметов искусства часто пишутся с большой буквы, и будет глупо прятать от модели такую информацию.

In [28]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

При токенизации слова могут разделиться на несколько токенов (как слово `Fischler` из примера ниже), из-за чего появится несоответствие между числом токенов и меток. Это несоответствие нам придется устранить вручную.

In [29]:
example = dataset["train"][12]
words = example["tokens"]
tags = [label_names[id] for id in example["ner_tags"]]
tokenized_text = tokenizer(example["tokens"], is_split_into_words=True)

print('Слова: ', words)
print('Токены:', tokenized_text.tokens())
print('Метки:', tags)

Слова:  ['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']
Токены: ['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']
Метки: ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'O', 'O', 'O']


__Задание 1 (1 балл).__ Токенизируйте весь датасет и для каждого текста выравните токены с метками так, чтобы каждому токену соответствовала одна метка. При этом важно сохранить нотацию BIO. И не забудьте про специальные токены! Должно получиться что-то такое:

In [31]:
from NLP.nlp_hw4.data_utils import align_labels_with_tokens_single

aligned_labels = align_labels_with_tokens_single(example["ner_tags"], tokenized_text)
tags = [label_names[t] if t > -1 else t for t in aligned_labels]
print("Выровненные метки:", aligned_labels)
print("Выровненные названия меток:", tags)

Выровненные метки: [-100, 0, 5, 0, 5, 0, 1, 2, 2, 0, 0, 0, 0, -100]
Выровненные названия меток: [-100, 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', -100]


Весь датасет:

In [33]:
from NLP.nlp_hw4.data_utils import prepare_dataset

processed_dataset = prepare_dataset(dataset, tokenizer, label_names, max_length=60)
processed_dataset

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

### Метрика

Для оценки качества NER обычно используют F1 меру с микро-усреднением. Мы загрузим ее из библиотеки `seqeval`. Функция `f1_score` принимает два 2d списка с правильными и предсказанными метками, записаными текстом, и возвращает для них значение F1. Вы можете использовать ее с параметрами по умолчанию.

In [35]:
from metrics import compute_metrics

Особенность подсчета F1 для NER заключается в том, что в некоторых ситуациях неправильные ответы могут засчитываться как правильные. Например, если модель предсказала `['I-PER', 'I-PER']`, то мы можем догадаться, что на самом деле должно быть `['B-PER', 'I-PER']`, так как сущность не может начинаться с `I-`. Функция `f1_score` учитывает это и поэтому работает только с текстовыми представлениями меток.

### Модель

В качестве базовой модели мы возьмем `bert-base-cased`. Как вы понимаете, он не обучался на задачу NER. Поэтому прежде чем приступать к уменьшению размера BERT, его необходимо дообучить.

__Задание 2 (1 балл)__ Дообучите `bert-base-cased` на нашем датасете с помощью обычного fine-tuning. У вас должно получиться хотя бы 0.9 F1 на тестовой выборке. Заметьте, что чем выше качество большой модели, тем лучше будет работать дистиллированный ученик. Для обучения можно использовать `Trainer` из Hugging Face.

In [36]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    'bert-base-cased',
    num_labels=len(label_names),
)

print('Число параметров:', sum(p.numel() for p in model.parameters()))

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Число параметров: 107726601


In [38]:
import wandb

wandb.login(key="26c846e74de0b390e85a23373ba9d37860c481cd")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [39]:
run_name = "bert_ft_5_test"
num_epochs = 15
batch_size = 32
steps_per_epoch = len(dataset["train"]) // batch_size - 1

In [ ]:
from model_utils import get_optimizer, get_scheduler

data_collator = DataCollatorForTokenClassification(tokenizer)
optimizer = get_optimizer(model)
scheduler = get_scheduler(optimizer, num_epochs, steps_per_epoch)

training_args = TrainingArguments(
    report_to="wandb",
    output_dir="nlp_distillation",
    run_name=run_name,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=1e-6,
    max_grad_norm=2.0,
    save_steps=50000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)
trainer.train()
wandb.finish()

На тесте:

In [ ]:
test_res = trainer.evaluate(eval_dataset=processed_dataset["test"])
print(test_res)

### Факторизация матрицы эмбеддингов

Можно заметить, что на данный момент матрица эмбеддингов занимает $V \cdot H = 28996 \cdot 768 = 22.268.928$ параметров. Это aж пятая часть от всей модели! Давайте попробуем что-то с этим сделать. В модели [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) предлагается факторизовать матрицу эмбеддингов в произведение двух небольших матриц. Таким образом, параметры эмбеддингов будут содержать $V \cdot E + E \cdot H$ элементов, что гораздо меньше $V \cdot H$, если $H \gg E$. Авторы выбирают $E = 128$, однако ничего не мешает нам взять любое другое значение. Например, выбрав $H = 64$, мы уменьшим число параметров примерно на 20М.

__Задание 3 (1 балл).__ Напишите класс-обертку над слоем эмбеддингов, который реализует факторизацию на две матрицы, и дообучите факторизованную модель. Заметьте, обе матрицы можно инициализировать с помощью SVD разложения, чтобы начальное приближение было хорошим. Это сэкономит очень много времени на дообучении. С рангом разложения $H = 64$ у вас должно получиться F1 больше 0.87.

In [ ]:
from models import FactorizedEmbedding

# на всякий случай копирую чтоб не въеbat' модель
model_ = copy.deepcopy(model)

source_embeds = model_.bert.embeddings.word_embeddings.weight.data.clone()

factor_size = 64 
num_embeddings = source_embeds.shape[0] #V
embedding_dim = source_embeds.shape[1] #H

factorized_embedding = FactorizedEmbedding(
    num_embeddings=num_embeddings,
    embedding_dim=embedding_dim,
    factor_size=factor_size,
    source_embeds=source_embeds
)

model_.bert.embeddings.word_embeddings = factorized_embedding
print('Число параметров:', sum(p.numel() for p in model_.parameters()))

In [ ]:
# run params
run_name = "albert_ft_3"
num_epochs = 20
batch_size = 32
steps_per_epoch = len(dataset["train"]) // batch_size - 1

data_collator = DataCollatorForTokenClassification(tokenizer)
optimizer = get_optimizer(model_)
scheduler = get_scheduler(optimizer, num_epochs, steps_per_epoch)

training_args = TrainingArguments(
    report_to="wandb",
    output_dir="nlp_distillation",
    run_name=run_name,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=1e-6,
    max_grad_norm=2.0,
    save_steps=50000,
)

trainer = Trainer(
    model=model_,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)
trainer.train()
wandb.finish()

In [ ]:
test_res = trainer.evaluate(eval_dataset=processed_dataset["test"])
print(test_res)

### Дистилляция знаний

Дистилляция знаний – это парадигма обучения, в которой знания модели-учителя дистиллируются в модель-ученика. Учеником может быть произвольная модель меньшего размера, решающая ту же задачу, однако обычно ученик имеет ту же архитектуру, что и учитель. При дистилляции используются два функционала ошибки:

1. Стандартная кросс-энтропия.
1. Функция, задающая расстояние между распределениями предсказаний учителя и ученика. Чаще всего используют KL-дивергенцию.

Для того, чтобы распределение предсказаний учителя не было вырожденным, к softmax добавляют температуру больше 1, например, 2 или 5.   
__Важно:__ при делении логитов на температуру значения градиентов уменьшаются в $\tau^2$ раз (проверьте это!). Поэтому для возвращения их в изначальный масштаб ошибку надо домножить на $\tau^2$. Подробнее об этом можно почитать в разделе 2.1 [оригинальной статьи](https://arxiv.org/pdf/1503.02531).

<img src="https://intellabs.github.io/distiller/imgs/knowledge_distillation.png" width="1000">

__Задание 4 (3 балла).__ Реализуйте метод дистилляции знаний, изображенный на картинке. Для подсчета ошибки между предсказаниями ученика и учителя используйте KL-дивергенцию [`nn.KLDivLoss(reduction="batchmean")`](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html) (обратите внимание на вормат ее входов). Для получения итоговой ошибки суммируйте мягкую ошибку с жесткой.   
В качестве учителя используйте дообученный BERT из задания 2. В качестве ученика возьмите необученную модель с размером __не больше 20M__ параметров. Вы можете использовать факторизацию матрицы эмбеддингов для уменьшения числа параметров. Если вы все сделали правильно, то на тестовой выборке вы должны получить значение F1 не меньше 0.7. Вам должно хватить примерно 20к итераций обучения для этого. Если у вас что-то не получается, то можно ориентироваться на статью про [DistilBERT](https://arxiv.org/abs/1910.01108) и на [эту статью](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT).

__Важно:__
* Не забывайте добавлять _warmup_ при обучении ученика.
* Не забывайте переводить учителя в режим _eval_.

In [ ]:
run_name = "bert_distil_3"
num_epochs = 100
batch_size = 32
steps_per_epoch = len(dataset["train"]) // batch_size - 1

# заводим препода
teacher_model = copy.deepcopy(model)
teacher_model.eval()

In [11]:
# заводим студента
config = DistilBertConfig.from_pretrained('distilbert-base-cased')

config.num_hidden_layers = 6
config.hidden_size = 384
config.num_labels = len(label_names)
student_model = DistilBertForTokenClassification(config)

# факторизуем слой эмбеддингов
source_embeds = student_model.distilbert.embeddings.word_embeddings.weight.data.clone()

factor_size = 64
num_embeddings = source_embeds.shape[0]
embedding_dim = source_embeds.shape[1]

factorized_embedding = FactorizedEmbedding(
    num_embeddings=num_embeddings,
    embedding_dim=embedding_dim,
    factor_size=factor_size,
    source_embeds=source_embeds
)

student_model.distilbert.embeddings.word_embeddings = factorized_embedding

print(sum(p.numel() for p in student_model.parameters()))

19815049


In [ ]:
from model_utils import DistilTrainer

data_collator = DataCollatorForTokenClassification(tokenizer)
optimizer = get_optimizer(student_model)
scheduler = get_scheduler(optimizer, num_epochs, steps_per_epoch)

training_args = TrainingArguments(
    report_to="wandb",
    output_dir="nlp_distillation",
    run_name=run_name,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=1e-6,
    max_grad_norm=2.0,
    warmup_steps=1000,
    save_steps=50000,
)


trainer = DistilTrainer(
    model=student_model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
    teacher_model=teacher_model,
    temperature=3.0,
)

trainer.train()
wandb.finish()

На тесте:

In [ ]:
test_results = trainer.evaluate(eval_dataset=processed_dataset["test"])
print(test_results)

# Задания на выбор

Как вы понимаете, есть еще довольно много разных способов уменьшить обученную модель. В этой секции вам предлагается реализовать разные техники на выбор. За каждую из них можно получить разное количество балов в зависимости от сложности. Успешность реализации будет оцениваться как по коду, так и по качеству на тестовой выборке. Все баллы за это дз, которые вы наберете сверх 10, будут считаться бонусными.   
В задании 4 вы обучали модель с ограничением числа параметров в 20М. При реализации техник из этой секции придерживайтесь такого же ограничение. Это позволит честно сравнивать методы между собой и делать правильные выводы. Напишите в отчете обо всем, что вы попробовали.

* __Шеринг весов (2 балла).__ В модификации BERT [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) помимо факторизации эмбеддингов предлагается шерить веса между слоями. То есть разные слои используют одни и те же веса. Такая техника эвивалентна применению одного и того же слоя несколько раз. Она позволяет в несколько раз уменьшить число параметров и не сильно потерять в качестве.
* __Факторизация промежуточных слоев (2 балла).__ Если можно факторизовать матрицу эмбеддингов, то и все остальное тоже можно. Для факторизации слоев существует много разных подходов и выбрать какой-то один сложно. Вы можете вдохновляться [этим списком](https://lechnowak.com/posts/neural-network-low-rank-factorization-techniques/), найти в интернете что-то другое или придумать метод самостоятельно. В любом случае в отчете обоснуйте, почему вы решили сделать так как сделали.
* __Приближение промежуточных слоев (2 балла).__ Мы обсуждали, что помимо приближения выходов модели ученика к выходам модели учителя, можно приближать выходы промежуточных слоев. В [этой работе](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) подробно написано, как это можно сделать.
* __Прунинг (4 балла).__ В методе [SparseGPT](https://arxiv.org/abs/2301.00774) предлагается подход, удаляющий веса модели один раз после обучения. При этом оказывается возможным удалить до половины всех весов без потери в качестве. Математика, стоящаяя за техникой, довольно сложная, однако общий подход простой – будем удалять веса в каждом слое по отдельности, при удалении части весов слоя, остальные веса будут перенастраиваться так, чтобы общий выход слоя не изменился.
* __Удаление голов (6 баллов).__ В данный момент мы используем все головы внимания, но ряд исследований показывает, что большинство из них можно выбросить без потери качества. В этой [статье](https://arxiv.org/pdf/1905.09418.pdf) предлагается подход, который добавляет гейты к механизму внимания, которые регулируют, какие головы участвуют в слое, а какие – нет. В процессе обучения гейты настраиваются так, чтобы большинство голов не использовалась. В конце обучения неиспользуемые головы можно удалить. За это задание дается много баллов, потому что в методе довольно сложная математика и подход плохо заводится. Если вы решитесь потратить на него свои силы, то в случае неудачи мы дадим промежуточные баллы, опираясь на отчет.   
__Совет:__ во время обучения внимательно следите за поведением гейтов. Если вы все сделали правильно, то они должны зануляться. Однако зануляются они не всегда сразу, им надо дать время и обучать модель подольше.

## Weight Sharing

In [22]:
from models import FactorizedEmbedding
from models import SharedAttentionBert

config = BertConfig.from_pretrained("bert-base-cased")
config.num_hidden_layers = 6
config.hidden_size = 648
config.num_labels = len(label_names)

shared_bert = SharedAttentionBert(config)

# факторизуем слойэмбеддингов
source_embeds = shared_bert.bert.embeddings.word_embeddings.weight.data.clone()
factor_size = 64
num_embeddings = source_embeds.shape[0]
embedding_dim = source_embeds.shape[1]

factorized_embedding = FactorizedEmbedding(
    num_embeddings=num_embeddings,
    embedding_dim=embedding_dim,
    factor_size=factor_size,
    source_embeds=source_embeds
)

# наглядно профит в параметрах:
print("Число параметров в старом эмбед слое:", sum(p.numel() for p in shared_bert.bert.embeddings.word_embeddings.parameters()))
print("Число параметров в новом эмбед слое:", sum(p.numel() for p in factorized_embedding.parameters()))
print("Число параметров в факторизованной модели:",
      sum(p.numel() for p in factorized_embedding.parameters()) + \
      sum(p.numel() for p in shared_bert.parameters()) - \
      sum(p.numel() for p in shared_bert.bert.embeddings.parameters()) - \
      shared_bert.param_savings
     )

shared_bert.bert.embeddings.word_embeddings = factorized_embedding

Число параметров в старом эмбед слое: 18789408
Число параметров в новом эмбед слое: 1897216
Число параметров в факторизованной модели: 19087009


In [ ]:
run_name = "shared_bert_factorized_2"
num_epochs = 80
batch_size = 32
steps_per_epoch = len(dataset["train"]) // batch_size - 1

data_collator = DataCollatorForTokenClassification(tokenizer)
optimizer = get_optimizer(shared_bert)
scheduler = get_scheduler(optimizer, num_epochs, steps_per_epoch)

training_args = TrainingArguments(
    report_to="wandb",
    output_dir="nlp_distillation",
    run_name=run_name,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=1e-6,
    max_grad_norm=2.0,
    save_steps=50000,
)

trainer = Trainer(
    model=shared_bert,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)
trainer.train()
wandb.finish()

## Layer Factorization

In [25]:
from models import FactorizedBertOutput, FactorizedBertIntermediate
from model_utils import replace_with_factorized_layers

config = BertConfig.from_pretrained('bert-base-cased')
config.num_hidden_layers = 8
config.hidden_size = 384
config.num_labels = len(label_names)
factor_bert = BertForTokenClassification(config)

k = 64

replace_with_factorized_layers(factor_bert, k)
print(sum(p.numel() for p in factor_bert.parameters()))

19683721


Как видим, удалось благодаря факторизации вставить побольше слоёв

## Pruning

Предобучаем модель:

In [ ]:
run_name = "tiny_bert_ft_1"
num_epochs = 30
batch_size = 32
steps_per_epoch = len(dataset["train"]) // batch_size - 1

tinybert = copy.deepcopy(model)

data_collator = DataCollatorForTokenClassification(tokenizer)
optimizer = get_optimizer(tinybert)
scheduler = get_scheduler(optimizer, num_epochs, steps_per_epoch)

training_args = TrainingArguments(
    report_to="wandb",
    output_dir="nlp_distillation",
    run_name=run_name,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=1e-6,
    max_grad_norm=2.0,
    save_steps=50000,
)

trainer = Trainer(
    model=tinybert,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)
trainer.train()
wandb.finish()

И теперь пруним:

In [ ]:
from models import Pruner
from model_utils import hook_fn

model_ = copy.deepcopy(tinybert)

captured_inputs = {}
captured_outputs = {}

hooks = []
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        hooks.append((name, module.register_forward_hook(hook_fn(name))))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch = processed_dataset['train'][:256]
input_ids = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)

model(input_ids, attention_mask=attention_mask)
for _, hook in hooks:
    hook.remove()

sparsity = 0.4
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        print(f"Pruning {name}")
        sparse_gpt = Pruner(module)
        inp = captured_inputs[name]
        out = captured_outputs[name]
        sparse_gpt.zero_weights(sparsity=sparsity, block_size=128)